In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

linewidth = 100
formatter = {
    'float': lambda num: f'{num:5.1f}',
    'int': lambda num: f'{float(num):5.1f}',
}
print_args = {
#     'threshold': sys.maxsize,
    'threshold': 100,
    'linewidth': 100,
    'formatter': formatter,
    'edgeitems': 6,
}

In [3]:
import numpy as np
from qtm.constants import RYDBERG
from qtm.lattice import RealLattice, ReciLattice

alat = 5.107
latvec_alat = 0.5 * np.array([
    [ 1,  1,  1],
    [-1,  1,  1],
    [-1, -1,  1]
])

ecutwfn = 40 * RYDBERG

reallat = RealLattice.from_alat(alat, *latvec_alat)
recilat = ReciLattice.from_reallat(reallat)

In [4]:
from qtm.gspace import GSpace

grho = GSpace(recilat, 4 * ecutwfn)
print(grho.grid_shape)
print(grho.size_g)
with np.printoptions(**print_args):
    for i in range(3):
        print(f'G_{i+1}: ', grho.g_cryst[i])
    print(f'G^2: ', grho.g_norm2)

(18, 18, 18)
2243
G_1:  [  0.0   0.0   0.0   0.0   0.0   0.0 ...  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0]
G_2:  [  0.0   0.0   0.0   0.0   0.0   0.0 ...  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0]
G_3:  [  0.0   1.0   2.0   3.0   4.0   5.0 ...  -6.0  -5.0  -4.0  -3.0  -2.0  -1.0]
G^2:  [  0.0   3.0  12.1  27.2  48.4  75.7 ... 112.0  78.7  51.5  30.3  15.1   6.1]


In [5]:
from qtm.gspace import GkSpace

gkwfn = GkSpace(grho, [0., 0., 0.5])
print(gkwfn.grid_shape)
print(gkwfn.size_g)
with np.printoptions(**print_args):
    for i in range(3):
        print(f'G_{i+1}: ', gkwfn.gk_cryst[i])
    print(f'G^2: ', gkwfn.gk_norm2)

(18, 18, 18)
276
G_1:  [  0.0   0.0   0.0   0.0   0.0   0.0 ...  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0]
G_2:  [  0.0   0.0   0.0   0.0   0.0   0.0 ...  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0]
G_3:  [  0.5   1.5   2.5   3.5  -3.5  -2.5 ...   0.5   1.5   2.5  -2.5  -1.5  -0.5]
G^2:  [  0.8   6.8  18.9  37.1  37.1  18.9 ...   3.8   9.8  21.9  21.9   9.8   3.8]


In [7]:
from qtm.containers import FieldG, FieldR

def print_info(f):
    typ = type(f)
    basis = getattr(f, 'basis_type', 'N\A')
    shape = getattr(f, 'shape', 'N\A')
    print(f"type: {str(type(f)):35}\nbasis: '{basis}', shape: {shape}\n")

# Creating a empty GField instance
print('empty')
a_g = FieldG.empty(grho, (4, 10))
print_info(a_g)

# Converting to RField instance
print('rfield')
a_r = a_g.to_r()
print_info(a_r)


empty
type: <class 'qtm.containers.field.FieldG'>
basis: 'g', shape: (4, 10)

rfield
type: <class 'qtm.containers.field.FieldR'>
basis: 'r', shape: (4, 10)



In [8]:
# Indexing
print('indexing')
arr_idx = a_r[2]
print_info(arr_idx)

# Slicing
print('slicing')
arr_sl = a_r[:, :5]
print_info(arr_sl)

indexing
type: <class 'qtm.containers.field.FieldR'>
basis: 'r', shape: (10,)

slicing
type: <class 'qtm.containers.field.FieldR'>
basis: 'r', shape: (4, 5)



In [9]:
# Unpacking
print('unpacking')
x, y = a_r[0, :2]
print_info(x)
print_info(y)

# Binary operations
print('add')
z = x + y
print_info(z)

# Binary operations with broadcasting
print('add bcast')
z = x + a_r[:2,:3]
print_info(z)

unpacking
type: <class 'qtm.containers.field.FieldR'>
basis: 'r', shape: ()

type: <class 'qtm.containers.field.FieldR'>
basis: 'r', shape: ()

add
type: <class 'qtm.containers.field.FieldR'>
basis: 'r', shape: ()

add bcast
type: <class 'qtm.containers.field.FieldR'>
basis: 'r', shape: (2, 3)



In [11]:
# Operations between mismatched basis; raises exception
print('mismatch')
try:
    z = x + x.to_g()
except TypeError as e:
    print(e)
## TypeError: mismatch in 'basis_type' between two Field instances

mismatch
mismatch in 'basis_type' between two 'Buffer' instances.


In [12]:
a_g = FieldG.empty(grho, (4, 10))
print_info(a_g)
a_g.g[:] = np.random.rand(*a_g.data.shape)

s = sum(a_g)
print_info(s)
print(np.allclose(s.data, np.sum(a_g.data, axis=0)))

s = sum(sum(a_g))
print_info(s)
print(np.allclose(s.data, np.sum(a_g.data, axis=(0, 1))))

type: <class 'qtm.containers.field.FieldG'>
basis: 'g', shape: (4, 10)

type: <class 'qtm.containers.field.FieldG'>
basis: 'g', shape: (10,)

True
type: <class 'qtm.containers.field.FieldG'>
basis: 'g', shape: ()

True


In [13]:
from qtm.containers import WavefunSpinG

print('empty')
a_g = WavefunSpinG.empty(gkwfn, (4, 10))
print_info(a_g)

empty
type: <class 'qtm.containers.wavefun.WavefunSpinG'>
basis: 'g', shape: (4, 10)



In [14]:
print(a_g.basis_size)
a_r = a_g.to_r()
print(a_r.shape, a_r.basis_size)

pot_r = FieldR.empty(a_g.gspc, 1)
print(pot_r.shape, pot_r.basis_size)

552
(4, 10) 11664
(1,) 5832
